In [31]:
class DateTime:
    days_list = ["SRJ", "LK", "BM"]
    duration_mapping = {"SRJ":50 , "LK": 70, "BM": 70}

    def __init__(self, start_time, days):
        self.start_time = start_time
        self.days = days
        self.duration = DateTime.duration_mapping.get(self.days)

    def __repr__(self):
        return "dt{%d, %s}" % (self.start_time, self.days)


class shift:

    def __init__(self, satpam_id, gedung_num, date_time):
        self.satpam_id = satpam_id
        self.gedung_num = gedung_num
        self.date_time = date_time

    def mutasi_satpam_id(self, satpam_id):
        self.satpam_id = satpam_id

    def mutasi_gedung(self, gedung_num):
        self.gedung_num = gedung_num

    def mutasi_date_time(self, date_time):
        self.date_time = date_time

    def __repr__(self):
        return "%s, %s, %s" % (self.satpam_id, self.gedung_num, self.date_time)

In [32]:
import sys
import pprint
import copy
import random


nama_satpam = ["Andi", "Tomi", "Joko", "Irman", "Budi", "Luhan", "Robi", "Tisna", "Putra", "Putri", "Dono",
           "Doni", "Dudung", "Dadang", "Soni", "Bima", "Haikal", "Jono", "Rusdi", "Lala", "Eko"]

gedungITERA = ["A", "B", "C", "E", "F", "G"] 

DAYS_OF_WEEK = "SLRKJBM"

GENOMES_SIZE = 5000

MIN_START_TIME = 8
MAX_START_TIME = 16

NUM_SATPAM = len(nama_satpam)
NUM_GEDUNG = len(gedungITERA)
TIME_SLICES = MAX_START_TIME - MIN_START_TIME + 2

SATPAM_INC = 2 
GEDUNG_INC = 1

TOP_PERCENT = .05
PERCENT_UN_MUTATED = .01

random.seed()

pp = pprint.PrettyPrinter(indent=2)


def fitness(gene):
    global nama_satpam
    score = SATPAM_INC * NUM_SATPAM + GEDUNG_INC * 2 * NUM_SATPAM + 1
    hitung_satpam = {}
    for satpam_id in nama_satpam:
        hitung_satpam[satpam_id] = 0
    hitung_gedung = {}
    for gedung in gedungITERA:
        hitung_gedung[gedung] = {}
        for day in DAYS_OF_WEEK:
            hitung_gedung[gedung][day] = []
            for i in range(MIN_START_TIME, MAX_START_TIME + 2, 1):
                hitung_gedung[gedung][day].append(0)
    for satpam in gene:
        hitung_satpam[satpam.satpam_id] += 1
        for time_slice in range(satpam.date_time.start_time - MIN_START_TIME,
                                satpam.date_time.start_time + satpam.date_time.duration // 60 + 1 - MIN_START_TIME):
            for day in satpam.date_time.days:
                hitung_gedung[satpam.gedung_num][day][time_slice] += 1
  
    for hitung in hitung_satpam.values():
        if hitung == 1:
            score += SATPAM_INC
        else:
            score -= SATPAM_INC
            
    for days in hitung_gedung.values():
        for day in days.values():
            for hitung in day:
                if hitung != 0 and hitung != 1:
                    score -= GEDUNG_INC
    if score <= 0:
        print("score of %d not allowed, exiting..." % score)
        sys.exit(-1)
    return score


def selection(genomes, scores):
    children = []
    # select top 10% to be part of children
    children.extend([genomes[i[0]] for i in scores[:(int(GENOMES_SIZE * TOP_PERCENT))]])

    scores_sum = sum([i[1] for i in scores])
    proportional_selection = []
    index = 0
    for i in scores:
        proportional_selection.append(range(index, index + i[1]))
        index = index + i[1]

    for i in range(0, GENOMES_SIZE - int(GENOMES_SIZE * TOP_PERCENT)):
        selection1 = random.randint(0, scores_sum - 1)
        selection2 = random.randint(0, scores_sum - 1)

        parent1 = None
        parent2 = None

        for k in range(len(proportional_selection)):
            if selection1 in proportional_selection[k]:
                parent1 = genomes[scores[k][0]]
            if selection2 in proportional_selection[k]:
                parent2 = genomes[scores[k][0]]
            if parent1 is not None and parent2 is not None:
                break

        point1 = random.randint(1, NUM_SATPAM - 2)
        point2 = random.randint(point1, NUM_SATPAM - 1)

        children.append(copy.deepcopy(parent1[:point1] + parent2[point1:point2] + parent1[point2:]))
    print("selection complete")
    return children


def mutasi(genomes, scores):
    global nama_satpam
    for i in range(int(GENOMES_SIZE * PERCENT_UN_MUTATED), GENOMES_SIZE):
        for c in range(NUM_SATPAM):
            if random.randint(1, NUM_SATPAM) == 1:
                hasil_mutasi = random.randint(0, 2)
                if hasil_mutasi == 0:
                    new_satpam_id = random.choice(nama_satpam)
                    genomes[scores[i][0]][c].mutasi_satpam_id(new_satpam_id)
                elif hasil_mutasi == 1:
                    new_gedung = random.choice(gedungITERA)
                    genomes[scores[i][0]][c].mutasi_gedung(new_gedung)
                elif hasil_mutasi == 2:
                    start_time = random.randint(MIN_START_TIME, MAX_START_TIME)
                    day_structure = random.choice(DateTime.days_list)
                    date_time = DateTime(start_time, day_structure)
                    genomes[scores[i][0]][c].mutasi_date_time(date_time)
    print("mutasi complete")


def main():
    genomes = []
    global satpam

    for gene in range(GENOMES_SIZE):
        genomes.append([])
        for i in range(NUM_SATPAM):
            satpam_id = random.choice(nama_satpam)
            gedung = random.choice(gedungITERA)
            start_time = random.randint(MIN_START_TIME, MAX_START_TIME)
            day_structure = random.choice(DateTime.days_list)
            date_time = DateTime(start_time, day_structure)
            genomes[gene].append(shift(satpam_id, gedung, date_time))
 
    for i in range(0, 50):
        scores = []
        for gene in range(GENOMES_SIZE):
            scores.append((gene, fitness(genomes[gene])))
        scores = sorted(scores, key=lambda x: x[1], reverse=True)
        print(scores[:10])
        print(scores[-10:])
        print("\n")
        genomes = selection(genomes, scores)

        scores = []
        for gene in range(GENOMES_SIZE):
            scores.append((gene, fitness(genomes[gene])))
        scores = sorted(scores, key=lambda x: x[1], reverse=True)
        print(scores[:10])
        print(scores[-10:])
        print("\n")
        mutasi(genomes, scores)
    pp.pprint(genomes[scores[0][0]])
    print("\n")
    hitung_gedung = {}
    for gedung in gedungITERA:
        hitung_gedung[gedung] = {}
        for day in DAYS_OF_WEEK:
            hitung_gedung[gedung][day] = []
            for i in range(MIN_START_TIME, MAX_START_TIME + 2, 1):
                hitung_gedung[gedung][day].append(0)
    for satpam in genomes[scores[0][0]]:
        for time_slice in range(satpam.date_time.start_time - MIN_START_TIME,
                                satpam.date_time.start_time + satpam.date_time.duration // 60 + 1 - MIN_START_TIME):
            for day in satpam.date_time.days:
                hitung_gedung[satpam.gedung_num][day][time_slice] += 1
    pp.pprint(hitung_gedung)


if __name__ == "__main__":
    main()

[(2833, 101), (4252, 101), (3363, 100), (991, 99), (1299, 99), (1405, 99), (4259, 99), (386, 97), (2464, 97), (931, 96)]
[(894, 42), (1949, 42), (132, 41), (1811, 41), (2140, 41), (2382, 41), (625, 40), (2415, 39), (4593, 38), (1956, 37)]


selection complete
[(0, 101), (1, 101), (2131, 101), (4151, 101), (2, 100), (4695, 100), (3, 99), (4, 99), (5, 99), (6, 99)]
[(4796, 42), (3303, 41), (3782, 41), (3940, 41), (2503, 40), (2022, 38), (4546, 38), (3908, 36), (605, 35), (3809, 35)]


mutasi complete
[(0, 101), (1, 101), (904, 101), (2131, 101), (2635, 101), (4151, 101), (2, 100), (4695, 100), (3, 99), (4, 99)]
[(3064, 42), (3318, 42), (3303, 41), (3782, 41), (3940, 41), (2503, 40), (2022, 38), (4546, 38), (3908, 36), (3809, 35)]


selection complete
[(340, 103), (0, 101), (1, 101), (2, 101), (3, 101), (4, 101), (5, 101), (313, 101), (4698, 101), (6, 100)]
[(2703, 43), (3206, 43), (4758, 43), (350, 42), (1195, 42), (2581, 42), (1688, 40), (4396, 40), (3717, 37), (4567, 37)]


mutasi comp

mutasi complete
[(656, 117), (0, 115), (1, 115), (2, 115), (3, 115), (4, 115), (5, 115), (6, 115), (7, 115), (8, 115)]
[(3309, 46), (311, 43), (822, 43), (1142, 43), (2355, 43), (2792, 43), (3794, 43), (4232, 42), (3379, 41), (4905, 32)]


selection complete
[(498, 119), (0, 117), (1, 115), (2, 115), (3, 115), (4, 115), (5, 115), (6, 115), (7, 115), (8, 115)]
[(4249, 47), (905, 45), (2588, 45), (3879, 45), (958, 44), (4576, 44), (2748, 43), (1214, 42), (1141, 41), (1815, 41)]


mutasi complete
[(498, 119), (3825, 119), (0, 117), (3055, 117), (1, 115), (2, 115), (3, 115), (4, 115), (5, 115), (6, 115)]
[(3938, 46), (796, 45), (2014, 45), (3879, 45), (958, 44), (4576, 44), (2748, 43), (1214, 42), (1141, 41), (1815, 41)]


selection complete
[(0, 119), (1, 119), (2, 117), (3, 117), (4, 115), (5, 115), (6, 115), (7, 115), (8, 115), (9, 115)]
[(3611, 45), (4512, 45), (4789, 45), (2448, 44), (3070, 44), (3731, 43), (4989, 43), (4822, 41), (3032, 38), (3927, 37)]


mutasi complete
[(0, 119), (

mutasi complete
[(0, 127), (1, 127), (2, 127), (3, 127), (884, 127), (3941, 127), (4, 125), (5, 125), (6, 125), (7, 125)]
[(2829, 48), (3484, 48), (3883, 47), (1483, 45), (3568, 44), (2789, 43), (2463, 42), (2620, 42), (3363, 39), (2589, 38)]


selection complete
[(0, 127), (1, 127), (2, 127), (3, 127), (4, 127), (5, 127), (303, 127), (480, 127), (788, 127), (1414, 127)]
[(4510, 49), (4825, 49), (3669, 48), (1663, 47), (1986, 47), (4905, 47), (456, 46), (3924, 45), (4613, 45), (3538, 43)]


mutasi complete
[(0, 127), (1, 127), (2, 127), (3, 127), (4, 127), (5, 127), (303, 127), (480, 127), (788, 127), (1414, 127)]
[(2022, 48), (3120, 48), (3669, 48), (1663, 47), (1986, 47), (3538, 47), (4905, 47), (456, 46), (3924, 45), (4613, 45)]


selection complete
[(0, 127), (1, 127), (2, 127), (3, 127), (4, 127), (5, 127), (6, 127), (7, 127), (8, 127), (9, 127)]
[(4617, 52), (1413, 51), (2985, 51), (3319, 51), (3894, 51), (4539, 51), (3009, 50), (4651, 50), (2302, 46), (3127, 45)]


mutasi comple